# Clasificación con regresión logística y árboles de decisión

Este notebook está dividido en tres partes, y tiene como objetivo explorar cómo construir y evaluar modelos de clasificación utilizando tres algoritmos fundamentales: regresión logística, árboles de decisión y random forest. Comenzaremos con una implementación desde cero del algoritmo de regresión logística, aplicado a la detección de cáncer de mama, y luego compararemos su desempeño con la versión integrada en scikit-learn. Posteriormente, utilizaremos árboles de decisión para modelar el mismo problema, y analizaremos cómo los hiperparámetros afectan su capacidad de generalización. Finalmente, emplearemos el algoritmo de random forest, una técnica de ensamblaje que combina múltiples árboles de decisión para mejorar la robustez y el rendimiento predictivo del modelo. A lo largo del notebook, haremos énfasis en buenas prácticas de validación cruzada, evaluación con métricas apropiadas (como el F1 score), y detección de posibles problemas de sobreajuste.

## Parte 1: Regresión Logística

La **regresión logística** es un modelo de clasificación supervisada que se utiliza para predecir probabilidades asociadas a clases binarias (por ejemplo, 0 o 1, Positivo o Negativo, Sí o No). A diferencia de la regresión lineal, la salida del modelo no es un número real continuo, sino una probabilidad entre 0 y 1.

Para este modelo escogemos la siguiente hipótesis que predice probabilidades de que puntos de datos pertenezcan a una clase:

$$
\hat{y} = \sigma(z) = \sigma(\mathbf{x}^\top \mathbf{w} + b)
$$

donde:

- $\hat{y}$ es la probabilidad predicha de pertenecer a la clase 1,
- $\mathbf{x}$ es el vector de características o features de entrada,
- $\mathbf{w}$ son los pesos del modelo,
- $b$ es el sesgo o bias,
- $\sigma(\cdot)$ es la **función sigmoide**, definida como:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Esta función transforma cualquier número real en el intervalo $(0, 1)$, permitiendo interpretarlo como una probabilidad.

La regresión logística se entrena minimizando la **entropía cruzada binaria** (también conocida como log-loss):

$$
J(\mathbf{w}, b) = - \frac{1}{m} \sum_{i=1}^{m} \left[ y^{(i)} \log \hat{y}^{(i)} + (1 - y^{(i)}) \log (1 - \hat{y}^{(i)}) \right]
$$

donde:

- $m$ es el número de ejemplos de entrenamiento,
- $y^{(i)}$ es la clase verdadera (0 o 1),
- $\hat{y}^{(i)}$ es la probabilidad predicha para la clase 1.



Para medir el desempeño de los modelos de clasifiación binaria, comunmente se usa la **matriz de confusión**: una tabla que resume el rendimiento del modelo. Se organiza de la siguiente forma:

|                       | Predicción: 0 | Predicción: 1 |
|-----------------------|---------------|---------------|
| **Real: 0**           | TN (verdaderos negativos) | FP (falsos positivos) |
| **Real: 1**           | FN (falsos negativos)     | TP (verdaderos positivos) |

- **TP (True Positives):** el modelo predijo 1 y era 1.  
- **TN (True Negatives):** el modelo predijo 0 y era 0.  
- **FP (False Positives):** el modelo predijo 1 pero era 0.  
- **FN (False Negatives):** el modelo predijo 0 pero era 1.

Para tener una medida única de rendimiento, se usa el **F1 score**. Esta es una métrica de rendimiento que combina la **precisión** (“¿De todas las veces que el modelo predijo la clase positiva (1), cuántas veces acertó?”) y la **recuperación** (“¿De todos los casos realmente positivos (1), cuántos detectó el modelo?”) en un solo número. Es especialmente útil cuando los datos están desequilibrados.

Se define como:

$$
\text{F1} = 2 \cdot \frac{\text{precisión} \cdot \text{recuperación}}{\text{precisión} + \text{recuperación}}
$$

donde:

- $\text{precisión} = \frac{TP}{TP + FP}$  
  (de todas las veces que el modelo predijo 1, ¿cuántas fueron correctas?),
  
- $\text{recuperación} = \frac{TP}{TP + FN}$  
  (de todos los verdaderos casos positivos, ¿cuántos encontró el modelo?).

Un valor de F1 cercano a 1 indica que el modelo tiene un buen equilibrio entre precisión y recuperación.


In [1]:
# Importamos librerías necesarias

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.pipeline import make_pipeline
import pandas as pd

In [2]:
# Cargamos el dataset de cancer de mama y lo convertimos a DataFrame de pandas

cancer = load_breast_cancer()
df_cancer = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df_cancer['target'] = cancer.target
df_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


**Importante: Generalmente es necesario hacer un análisis detallado de todos los datos, ver sus estadísticas, graficarlos, chequear que los datos estén balanceados. También es buena práctica entender a fondo el dataset y sus características para identificar características redundantes. Esto no lo haremos hoy aquí** 

In [3]:
# EJERCICIO: Usa los métodos de pandas para ver explorar el DataFrame
#df_cancer.describe()
#df_cancer.info()
#df_cancer.shape
#df_cancer.dtypes.unique()
#df_cancer.isnull()
#df_cancer.isnull().sum()
#df_cancer['target'].value_counts()


In [4]:
class LogReg:
    """
    Implementación personalizada de regresión logística usando descenso de gradiente.
    """
    def __init__(self):
        """
        Inicializa los parámetros del modelo.
        """
        self.w = None 
        self.b = None 
    
    @staticmethod
    def sigmoid(z):
        """
        Calcula la función sigmoide.
        
        Args:
            z (numpy.ndarray): Arreglo de entrada.
        
        Returns:
            numpy.ndarray: Resultado de aplicar la sigmoide elemento por elemento.
        """
        return 1 / (1 + np.exp(-z)) 
    
    def entrenar(self, x_train, y_train, alpha, iter=1000):
        """
        Entrena el modelo de regresión logística usando descenso de gradiente.
        
        Args:
            x_train (numpy.ndarray): Características de entrenamiento.
            y_train (numpy.ndarray): Etiquetas de entrenamiento.
            alpha (float): Tasa de aprendizaje.
            iter (int): Número de iteraciones.
        
        Returns:
            tuple: (w, b) Vector de pesos y sesgo optimizados.
        """
        x_train = np.array(x_train)
        y_train = np.array(y_train).flatten()

        if x_train.ndim == 1:
            x_train = x_train.reshape(-1, 1) 

        m, n = x_train.shape 
        w = np.zeros(n)  
        b = 0. 

        for _ in range(iter):
            z = x_train @ w + b 
            h = LogReg.sigmoid(z)  
            dcost_dw = (1 / m) * (x_train.T @ (h - y_train))
            dcost_db = (1 / m) * np.sum(h - y_train)
            w -= alpha * dcost_dw
            b -= alpha * dcost_db

        self.w = w 
        self.b = b
        return self.w, self.b
    
    def predecir(self, x):
        """
        Predice etiquetas de clase usando el modelo entrenado.
        
        Args:
            x (numpy.ndarray): Datos de entrada.
        
        Returns:
            numpy.ndarray: Etiquetas predichas (0 o 1).
        """
        probabilidades = LogReg.sigmoid(x @ self.w + self.b)  
        predicciones = probabilidades > 0.5  
        return predicciones.astype(int) 


In [5]:
# Preparamos los datos para el modelo

X_cancer = df_cancer.drop(columns=['target']).to_numpy()
y_cancer = df_cancer['target'].to_numpy()

# Dividimos los datos en conjuntos de entrenamiento y prueba

X_cancer_train, X_cancer_test, y_cancer_train, y_cancer_test = train_test_split(X_cancer, y_cancer, test_size=0.2, random_state=42)

# Escalamos los datos

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_cancer_train)
X_train_scaled = scaler.transform(X_cancer_train)
X_test_scaled = scaler.transform(X_cancer_test)

In [6]:
# Creamos una instancia del modelo de regresión logística y lo entrenamos con los datos escalados

mi_modelo = LogReg()
mi_modelo.entrenar(X_train_scaled, y_cancer_train, alpha=0.01, iter=1000)

(array([-0.22049866,  0.01786638, -0.23164502, -0.23792155,  0.13485344,
        -0.18186592, -0.33299901, -0.38794065,  0.07775716,  0.23141199,
        -0.13102749,  0.1519092 , -0.12003654, -0.1252357 ,  0.17053353,
         0.01011835,  0.00337892,  0.01117037,  0.19063616,  0.07079685,
        -0.30010121, -0.01648601, -0.29387164, -0.26403418,  0.06103145,
        -0.19823976, -0.25359031, -0.38245854, -0.00939993,  0.00667793]),
 np.float64(0.8630419169674644))

In [7]:
# Hacemos predicciones y evaluamos el modelo

mis_predicciones = mi_modelo.predecir(X_test_scaled)
cm = confusion_matrix(y_cancer_test, mis_predicciones)
print(f'\n=== Tenemos un accuracy de: {accuracy_score(y_cancer_test, mis_predicciones):.2f} ===\n'
      f'\n== Tenemos {cm[0, 0]} verdaderos negativos ==\n'
      f'\n== Tenemos {cm[1, 1]} verdaderos positivos ==\n'
      f'\n== Tenemos {cm[0, 1]} falsos positivos ==\n'   
      f'\n== Tenemos {cm[1, 0]} falsos negativos ==\n'   
      f'\n== Tenemos un F1 Score de: {f1_score(y_cancer_test, mis_predicciones):.2f} ==\n')



=== Tenemos un accuracy de: 0.93 ===

== Tenemos 35 verdaderos negativos ==

== Tenemos 71 verdaderos positivos ==

== Tenemos 8 falsos positivos ==

== Tenemos 0 falsos negativos ==

== Tenemos un F1 Score de: 0.95 ==



¡Esto demuestra un buen desempeño del modelo! (Recordemos que es buena práctica realizar validación cruzada. ¡Lo haremos más adelante con el modelo propio de `scikit-lear`!)

 Ahora comparémoslo con el de `scikit-learn`.

In [8]:
modelo_sklearn = make_pipeline(MinMaxScaler(feature_range=(0, 1)), LogisticRegression())
modelo_sklearn.fit(X_cancer_train, y_cancer_train)
predicciones_sklearn = modelo_sklearn.predict(X_cancer_test)

In [9]:
# Evaluamos el modelo de sklearn

cm_sklearn = confusion_matrix(y_cancer_test, predicciones_sklearn)
print(f'\n=== sklearn tiene un accuracy de: {accuracy_score(y_cancer_test, predicciones_sklearn):.2f} ===\n'
      f'\n== sklearn tiene {cm_sklearn[0, 0]} verdaderos negativos ==\n'
      f'\n== sklearn tiene {cm_sklearn[1, 1]} verdaderos positivos ==\n'
      f'\n== sklearn tiene {cm_sklearn[0, 1]} falsos positivos ==\n'   
      f'\n== sklearn tiene {cm_sklearn[1, 0]} falsos negativos ==\n'   
      f'\n== sklearn tiene un F1 Score de: {f1_score(y_cancer_test, predicciones_sklearn):.2f} ==\n')


=== sklearn tiene un accuracy de: 0.98 ===

== sklearn tiene 41 verdaderos negativos ==

== sklearn tiene 71 verdaderos positivos ==

== sklearn tiene 2 falsos positivos ==

== sklearn tiene 0 falsos negativos ==

== sklearn tiene un F1 Score de: 0.99 ==



El modelo es sospechosamente bueno. Investiguemos su capacidad de generalización para chequear que no haya overfitting.

In [10]:
# Evaluamos el desempeño del modelo

X_cancer_temp, X_cancer_test, y_cancer_temp, y_cancer_test = train_test_split(X_cancer, y_cancer, test_size=0.2, random_state=42)

modelo_sklearn_cv = make_pipeline(MinMaxScaler(feature_range=(0, 1)), LogisticRegression())

scores = cross_val_score(modelo_sklearn_cv, X_cancer_temp, y_cancer_temp, cv=5, scoring='f1')
print("F1 scores para desarrollo:", scores)
print("F1 promedio:", scores.mean())
print("Desviación estándar de F1:", scores.std())

modelo_sklearn_cv.fit(X_cancer_temp, y_cancer_temp)
final_f1 = f1_score(y_cancer_test, modelo_sklearn_cv.predict(X_cancer_test))
print("F1 FINAL en conjunto de prueba:", final_f1)

F1 scores para desarrollo: [0.96610169 0.96610169 0.98245614 0.95798319 0.95798319]
F1 promedio: 0.9661251833472015
Desviación estándar de F1: 0.008936277353184214
F1 FINAL en conjunto de prueba: 0.9861111111111112


¡El modelo generaliza muy bien!

**Buenas prácticas en el desarrollo de modelos de machine learning:**

1. **Fase exploratoria (opcional):**  
   Realiza pruebas rápidas con un solo conjunto de entrenamiento/prueba si necesitas depurar código o verificar que los modelos funcionan.  
   No uses esta fase para tomar decisiones sobre qué modelo es mejor. Los resultados no son confiables ni representativos.

2. **Fase de selección de modelos (validación cruzada):**  
   Utiliza **validación cruzada** (por ejemplo, k-fold CV) en el conjunto de entrenamiento para:
   - Comparar modelos y elegir el mejor,
   - Ajustar hiperparámetros de forma objetiva,
   - Medir el rendimiento esperable en datos nuevos,
   sin haber tocado el conjunto de prueba.

3. **Evaluación final:**  
   Una vez elegido el modelo final, **entrena con todos los datos de entrenamiento** (sin CV)  
   y **evalúalo una única vez** en el conjunto de prueba completamente separado.  
   Esta evaluación representa el rendimiento real del modelo en producción.

**Resumen:** No utilices el conjunto de prueba para guiar decisiones durante el desarrollo.  
Solo úsalo al final para obtener una estimación honesta del rendimiento.



## Parte 2: Árboles de Decisión

Un **árbol de decisión** es un modelo de clasificación supervisada que predice la clase de una muestra a partir de una secuencia de preguntas binarias sobre sus características. La estructura del árbol está formada por **nodos de decisión**, donde se hace una pregunta del tipo “¿feature $x_j$ < umbral?”, y **hojas**, que contienen una predicción final (por ejemplo, clase 0 o clase 1).


Durante el entrenamiento, el algoritmo construye el árbol de la siguiente forma: en cada nodo busca la característica y el umbral que mejor separen las clases. Es decir, quiere dividir los datos en subconjuntos que sean lo más puros posible (que contengan principalmente una sola clase). Para evaluar qué tan buena es una división, el algoritmo puede usar uno de dos **criterios de impureza**:

La **entropía** mide el grado de incertidumbre de una distribución de clases en un conjunto de datos:

$$
H(D) = - \sum_{k=1}^{K} p_k \log_2(p_k)
$$

donde $p_k$ es la proporción de ejemplos de la clase $k$ en el conjunto $D$.

La **ganancia de información** al hacer una división se define como:

$$
\text{Gain}(D, \text{split}) = H(D) - \sum_{i=1}^{2} \frac{|D_i|}{|D|} H(D_i)
$$

donde $D_1$ y $D_2$ son los subconjuntos después de la división. Este valor mide cuánto se reduce la entropía gracias a la división.


Y la impureza según el **índice de Gini**, definido como:

$$
G(D) = 1 - \sum_{k=1}^{K} p_k^2
$$

donde, nuevamente, $p_k$ es la proporción de ejemplos de clase $k$. Un valor cercano a 0 indica alta pureza (una sola clase dominante).


El algoritmo de entrenamiento realiza los siguientes pasos:

1. Comienza con todos los datos en la raíz del árbol.
2. Busca la mejor división (característica y umbral) según el criterio escogido.
3. Divide los datos y crea nodos hijos.
4. Repite el proceso recursivamente para cada nodo hijo.
5. Se detiene cuando:
   - Los nodos contienen solo ejemplos de una clase,
   - Se alcanza una profundidad máxima,
   - O hay pocas muestras para continuar dividiendo.




In [11]:
# Importamos libreías necesarias

import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [12]:
# Cargamos el dataset de pingüinos de seaborn y lo convertimos a DataFrame de pandas

df_penguins = sns.load_dataset('penguins').dropna()
df_penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


In [13]:
df_penguins.shape

(333, 7)

In [14]:
# Analizamos la variable objetivo

print(df_penguins['sex'].unique())
print(df_penguins['sex'].value_counts())
print(df_penguins.dtypes.unique)

['Male' 'Female']
sex
Male      168
Female    165
Name: count, dtype: int64
<bound method Series.unique of species               object
island                object
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                   object
dtype: object>


¡Los datos están balanceados!

In [15]:
# Codificamos las características categóricas

categorical_cols = df_penguins.select_dtypes(include=['object', 'category']).columns
categorical_cols.drop('sex')

Index(['species', 'island'], dtype='object')

In [16]:
# Codificamos la variable objetivo
df_penguins['sex'] = df_penguins['sex'].map({'Female': 0, 'Male': 1})
df_penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,1
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,0
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,0
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,0
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,1


In [17]:
# Preparamos los datos para el modelo

y_penguins = df_penguins['sex'].to_numpy()
X_t = df_penguins.drop(columns=['sex'])
X_t = pd.get_dummies(X_t, drop_first=True) # Codificamos las características categóricas usando one-hot encoding
X_penguins = X_t.to_numpy()
X_penguins_temp, X_penguins_test, y_penguins_temp, y_penguins_test = train_test_split(X_penguins, y_penguins, test_size=0.2, random_state=42)
X_t.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,species_Chinstrap,species_Gentoo,island_Dream,island_Torgersen
0,39.1,18.7,181.0,3750.0,False,False,False,True
1,39.5,17.4,186.0,3800.0,False,False,False,True
2,40.3,18.0,195.0,3250.0,False,False,False,True
4,36.7,19.3,193.0,3450.0,False,False,False,True
5,39.3,20.6,190.0,3650.0,False,False,False,True


In [18]:
# Creamos y evaluamos el modelo

modelo_arbol = DecisionTreeClassifier(
    criterion='gini',
    max_depth=3,              # profundidad máxima del árbol
    min_samples_split=10,     # mínimo de muestras para dividir un nodo
    min_samples_leaf=5,       # mínimo de muestras por hoja
    random_state=42
)

# Validación cruzada sobre el conjunto de desarrollo
scores = cross_val_score(modelo_arbol, X_penguins_temp, y_penguins_temp, cv=5, scoring='f1')
print("F1 scores en validación cruzada:", scores)
print("F1 promedio (desarrollo):", scores.mean())
print("Desviación estándar:", scores.std())

# Entrenar el modelo final con todos los datos de desarrollo
modelo_arbol.fit(X_penguins_temp, y_penguins_temp)

# Paso 5: Evaluar una única vez en el conjunto de prueba
y_pred = modelo_arbol.predict(X_penguins_test)
final_f1 = f1_score(y_penguins_test, y_pred)
print("F1 FINAL en conjunto de prueba:", final_f1)


F1 scores en validación cruzada: [0.88888889 0.89655172 0.84210526 0.88135593 0.89285714]
F1 promedio (desarrollo): 0.8803517902490494
Desviación estándar: 0.019776293336427497
F1 FINAL en conjunto de prueba: 0.84375


Afinemos los hiperpámetros del modelo para ver si encontramos unos que mejoren el desempeño de manera significativa.

In [19]:
# Definimos el espacio de búsqueda
param_grid = {
    'criterion': ['gini', 'entropy'],  # Gini vs Información
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10]
}

# Usamos el mismo clasificador base
modelo_base = DecisionTreeClassifier(random_state=42)

# Definimos la búsqueda
grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

# Ejecutamos la búsqueda sobre el conjunto de desarrollo
grid_search.fit(X_penguins_temp, y_penguins_temp)

# Mostramos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluamos el mejor modelo en el conjunto de prueba
mejor_modelo = grid_search.best_estimator_
f1_final = f1_score(y_penguins_test, mejor_modelo.predict(X_penguins_test))
print("F1 FINAL del mejor modelo en el conjunto de prueba:", f1_final)

Mejores hiperparámetros encontrados:
{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 10}
F1 FINAL del mejor modelo en el conjunto de prueba: 0.819672131147541


**Comparación de modelos y capacidad de generalización**

Al comparar un modelo ajustado manualmente con hiperparámetros conservadores y otro optimizado mediante `GridSearchCV`, es importante no confundir "mejor validación cruzada" con "mejor capacidad de generalización".



**Definición clave**

> **Capacidad de generalización**: Es la habilidad de un modelo para mantener un rendimiento consistente al enfrentarse a datos nuevos y no vistos durante el entrenamiento o la validación.


**Resultados comparados**

| Métrica                      | Modelo conservador | Modelo GridSearch |
|-----------------------------|--------------------|-------------------|
| **F1 promedio (CV)**         | 0.880              | ~0.82 (estimado)  |
| **F1 final (test)**          | 0.844              | 0.820             |
| **Brecha de generalización** | 0.036              | ≈0.00             |


**Interpretación**

- Aunque el modelo de GridSearch parece tener una "brecha" menor entre CV y test, **esto no implica que generalice mejor**.
- De hecho, el modelo de GridSearch:
  - Tuvo **peor rendimiento en validación** y en test,
  - Probablemente **sobreajustó** los folds de validación cruzada,
  - Escogió hiperparámetros más flexibles (ej. `min_samples_leaf=1`), lo que aumentó el riesgo de sobreajuste.


**Conclusión**

> El modelo manual, con hiperparámetros conservadores, logró **mejor rendimiento final** y mostró una **mayor capacidad de generalización**.  
>  
> Este es un ejemplo clásico de cómo una estrategia de regularización simple y prudente puede superar a una optimización agresiva, especialmente cuando el volumen de datos es limitado.



**Consejo final:** No solo se debe mirar la métrica final — analiza también cómo varía entre entrenamiento, validación y prueba. ¡La estabilidad es clave para modelos confiables!


¿Tenemos otras opciones para mejorar el desempeño de nuestro modelo basado en árboles de decisión?

**¡Sí! Podemos usar un modelo de _Random Forest_**

El modelo de **Random Forest** es una técnica de **ensamble**, es decir, combina **muchos árboles de decisión** en lugar de usar solo uno. En lugar de confiar en un único árbol (que puede sobreajustarse o ser inestable), el bosque genera múltiples árboles y **promedia sus predicciones**. Funciona asíÑ

1. **Bootstrap:** Crea muchas muestras aleatorias del conjunto de entrenamiento original (con reemplazo).
2. **Entrena muchos árboles:** Cada árbol se entrena sobre una muestra diferente, usando solo un subconjunto aleatorio de características en cada división.
3. **Combina predicciones:** En clasificación, cada árbol vota y se toma la clase más votada; en regresión, se hace un promedio.

Las ventajas incluyen:

- **Reduce el sobreajuste:** Al promediar muchos árboles diferentes, se suavizan los errores de modelos individuales.
- **Mejor generalización:** La variación entre los árboles mejora la robustez frente a nuevos datos.

En resumen:

> **Random Forest = muchos árboles débiles → un modelo fuerte, más preciso y más estable.**


In [20]:
X_temp_rf, X_test_rf, y_temp_rf, y_test_rf = train_test_split(X_penguins, y_penguins, test_size=0.2, random_state=42)

In [21]:
# Modelo de random forest

modelo_bosque = RandomForestClassifier(
    n_estimators=100,         # número de árboles en el bosque
    max_depth=5,              # profundidad máxima de cada árbol
    min_samples_split=10,     # mínimo de muestras para dividir un nodo
    min_samples_leaf=5,       # mínimo de muestras en una hoja
    random_state=42,
    n_jobs=-1                 # usar todos los núcleos disponibles
)

# Paso 3: Validación cruzada sobre el conjunto de desarrollo
scores_rf = cross_val_score(modelo_bosque, X_temp_rf, y_temp_rf, cv=5, scoring='f1')
print("F1 scores en validación cruzada:", scores_rf)
print("F1 promedio (desarrollo):", scores_rf.mean())
print("Desviación estándar:", scores_rf.std())

modelo_bosque.fit(X_temp_rf, y_temp_rf)

y_pred_rf = modelo_bosque.predict(X_test_rf)
final_f1_rf = f1_score(y_test_rf, y_pred_rf)
print("F1 FINAL en conjunto de prueba:", final_f1_rf)


F1 scores en validación cruzada: [0.94736842 0.81632653 0.89655172 0.89285714 0.94545455]
F1 promedio (desarrollo): 0.8997116728228992
Desviación estándar: 0.04769408316900793
F1 FINAL en conjunto de prueba: 0.8709677419354839


Tratemos de mejorar el modelo con `GridSearchCV`.

In [22]:
param_grid = {
    'max_depth': [3, 5, 7, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2', None]
}

modelo_base = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)

grid_search = GridSearchCV(
    estimator=modelo_base,
    param_grid=param_grid,
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_temp_rf, y_temp_rf)

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

mejor_modelo = grid_search.best_estimator_
final_f1 = f1_score(y_test_rf, mejor_modelo.predict(X_test_rf))
print("F1 FINAL del mejor modelo en el conjunto de prueba:", final_f1)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Mejores hiperparámetros encontrados:
{'criterion': 'entropy', 'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2}
F1 FINAL del mejor modelo en el conjunto de prueba: 0.8709677419354839
